In [ ]:
# !pip install transformers

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import gc, warnings, random, time, os

from pathlib import Path

from tqdm.notebook import tqdm

warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import Adam, lr_scheduler
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
from transformers import AutoModel, AutoTokenizer, AutoConfig
from transformers import get_cosine_schedule_with_warmup

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

import seaborn as sns

import gc
gc.enable()

### Folders and Dataframes

In [2]:
DATA_PATH = Path('/home/commonlit/data/')
assert DATA_PATH.exists()
MODELS_PATH = Path('/home/commonlit/models/')
if not MODELS_PATH.exists():
    os.mkdir(MODELS_PATH)
assert MODELS_PATH.exists()

In [3]:
train_df = pd.read_csv(DATA_PATH/'train-orig.csv')
test_df = pd.read_csv(DATA_PATH/'test.csv')
sample_df = pd.read_csv(DATA_PATH/'sample_submission.csv')

In [4]:
def remove_unnecessary(df):
    df.drop(df[df['target'] == 0].index, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
remove_unnecessary(train_df)

In [5]:
train_df

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845
...,...,...,...,...,...,...
2828,25ca8f498,https://sites.ehe.osu.edu/beyondpenguins/files...,CC BY-SA 3.0,When you think of dinosaurs and where they liv...,1.711390,0.646900
2829,2c26db523,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,So what is a solid? Solids are usually hard be...,0.189476,0.535648
2830,cd19e2350,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,The second state of matter we will discuss is ...,0.255209,0.483866
2831,15e2e9e7a,https://en.wikibooks.org/wiki/Geometry_for_Ele...,CC BY-SA 3.0,Solids are shapes that you can actually touch....,-0.215279,0.514128


### Config and Seeding

In [6]:
class Config(): 
    NUM_FOLDS = 6
    NUM_EPOCHS = 3
    BATCH_SIZE = 16
    MAX_LEN = 248
    EVAL_SCHEDULE = [(0.50, 16), (0.49, 8), (0.48, 4), (0.47, 2), (-1., 1)]
    MODEL_PATH = 'facebook/bart-large'
    TOKENIZER_PATH = 'facebook/bart-large'
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    SEED = 1000
    NUM_WORKERS = 2
    MODEL_FOLDER = MODELS_PATH
    model_name = 'bart-large'
    svm_kernels = ['rbf']
    svm_c = 5

cfg = Config()

In [7]:
if not cfg.MODEL_FOLDER.exists():
    os.mkdir(cfg.MODEL_FOLDER)

In [8]:
def set_random_seed(random_seed):
    random.seed(random_seed)
    np.random.seed(random_seed)
    os.environ["PYTHONHASHSEED"] = str(random_seed)

    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)

    torch.backends.cudnn.deterministic = True

### Dataset

In [9]:
def add_bins(train_df, num_bins):
    train_df.loc[:, 'bins'] = pd.cut(train_df['target'], bins=num_bins, labels=False)
    return num_bins

In [10]:
add_bins(train_df, cfg.NUM_FOLDS)

6

In [11]:
train_df.groupby(['bins'])['target'].agg(['count', 'mean'])

,count,mean
bins,,
0,122,-3.125765
1,441,-2.270279
2,784,-1.412150
3,886,-0.548095
4,494,0.289716
5,106,1.070237


In [12]:
tokenizer = AutoTokenizer.from_pretrained(cfg.TOKENIZER_PATH)

In [13]:
class CommonLitDataset(Dataset):
    def __init__(self, df, tokenizer, inference_only=False):
        super().__init__()
        self.df, self.inference_only = df, inference_only
        self.text = df['excerpt'].tolist()
        self.bins = df['bins']
        if not inference_only:
            self.target = torch.tensor(df['target'].to_numpy(), dtype = torch.float32)
        
        self.encoded = tokenizer.batch_encode_plus(
            self.text,
            padding = 'max_length',
            max_length = cfg.MAX_LEN,
            truncation = True,
            return_attention_mask=True
        )
        
    def __getitem__(self, index):        
        input_ids = torch.tensor(self.encoded['input_ids'][index])
        attention_mask = torch.tensor(self.encoded['attention_mask'][index])
        
        if self.inference_only:
            return {'input_ids': input_ids, 'attention_mask': attention_mask}
        else:
            target = self.target[index]
            return {'input_ids': input_ids, 'attention_mask': attention_mask, 'target': target}
    
    def __len__(self):
        return len(self.df)

In [14]:
sample_ds = CommonLitDataset(train_df, tokenizer)

### Model

In [15]:
class AttentionHead(nn.Module):
    
    def __init__(self, in_features, hidden_dim, num_targets):
        super().__init__()
        self.in_features = in_features
        
        self.hidden_layer = nn.Linear(in_features, hidden_dim)
        self.final_layer = nn.Linear(hidden_dim, num_targets)
        self.out_features = hidden_dim
        
    def forward(self, features):
        att = torch.tanh(self.hidden_layer(features))
        score = self.final_layer(att)
        attention_weights = torch.softmax(score, dim=1)
        return attention_weights

In [16]:
class CommonLitModel(nn.Module):
    def __init__(self):
        super(CommonLitModel, self).__init__()
        config = AutoConfig.from_pretrained(cfg.MODEL_PATH)
        config.update({
            "output_hidden_states": True,
            "hidden_dropout_prob": 0.0,
            "layer_norm_eps": 1e-7
        })
        self.transformer_model = AutoModel.from_pretrained(cfg.MODEL_PATH, config=config)
        self.attention = AttentionHead(config.hidden_size, 512, 1)
        self.regressor = nn.Linear(config.hidden_size, 1)
    
    def forward(self, input_ids, attention_mask):
        last_layer_hidden_states = self.transformer_model(input_ids=input_ids, attention_mask=attention_mask)['last_hidden_state']
        weights = self.attention(last_layer_hidden_states)
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1) 
        return self.regressor(context_vector), context_vector

In [17]:
sample_model = CommonLitModel()

In [18]:
import re

current_layer = ''
base_lr = 5e-5
layer_map = {}
for i, (name, param) in enumerate(sample_model.named_parameters()):
    if(name.find('.layer') > -1):
        layer_name = re.sub(r'.+\.(layer\.\d+).+', r'\1', name)
        if(current_layer != layer_name):
            current_layer = layer_name
            layer_map[i] = current_layer

In [19]:
layer_lrs = []
for layer in layer_map.keys():
    if (layer < 170):
        layer_lrs.append((layer, base_lr / 5.0))
    elif (layer < 334):
        layer_lrs.append((layer, base_lr))
    else:
        layer_lrs.append((layer, base_lr * 20))

In [20]:
layer_lrs

[(2, 1e-05),
 (3, 1e-05),
 (4, 1e-05),
 (5, 1e-05),
 (6, 1e-05),
 (7, 1e-05),
 (8, 1e-05),
 (9, 1e-05),
 (10, 1e-05),
 (11, 1e-05),
 (12, 1e-05),
 (13, 1e-05),
 (14, 1e-05),
 (15, 1e-05),
 (16, 1e-05),
 (17, 1e-05),
 (18, 1e-05),
 (19, 1e-05),
 (20, 1e-05),
 (21, 1e-05),
 (22, 1e-05),
 (23, 1e-05),
 (24, 1e-05),
 (25, 1e-05),
 (26, 1e-05),
 (27, 1e-05),
 (28, 1e-05),
 (29, 1e-05),
 (30, 1e-05),
 (31, 1e-05),
 (32, 1e-05),
 (33, 1e-05),
 (34, 1e-05),
 (35, 1e-05),
 (36, 1e-05),
 (37, 1e-05),
 (38, 1e-05),
 (39, 1e-05),
 (40, 1e-05),
 (41, 1e-05),
 (42, 1e-05),
 (43, 1e-05),
 (44, 1e-05),
 (45, 1e-05),
 (46, 1e-05),
 (47, 1e-05),
 (48, 1e-05),
 (49, 1e-05),
 (50, 1e-05),
 (51, 1e-05),
 (52, 1e-05),
 (53, 1e-05),
 (54, 1e-05),
 (55, 1e-05),
 (56, 1e-05),
 (57, 1e-05),
 (58, 1e-05),
 (59, 1e-05),
 (60, 1e-05),
 (61, 1e-05),
 (62, 1e-05),
 (63, 1e-05),
 (64, 1e-05),
 (65, 1e-05),
 (66, 1e-05),
 (67, 1e-05),
 (68, 1e-05),
 (69, 1e-05),
 (70, 1e-05),
 (71, 1e-05),
 (72, 1e-05),
 (73, 1e-05),


In [21]:
def find_lr_for_layer(layer_num, layer_lrs):
    lr = layer_lrs[0][1]
    for layer_lr in layer_lrs:
        if layer_num >= layer_lr[0]:
            lr = layer_lr[1]
        else:
            break
    return lr

In [22]:
for i, (name, param) in enumerate(sample_model.named_parameters()):
    print(i, name, param.size())

0 transformer_model.shared.weight torch.Size([50265, 1024])
1 transformer_model.encoder.embed_positions.weight torch.Size([1026, 1024])
2 transformer_model.encoder.layers.0.self_attn.k_proj.weight torch.Size([1024, 1024])
3 transformer_model.encoder.layers.0.self_attn.k_proj.bias torch.Size([1024])
4 transformer_model.encoder.layers.0.self_attn.v_proj.weight torch.Size([1024, 1024])
5 transformer_model.encoder.layers.0.self_attn.v_proj.bias torch.Size([1024])
6 transformer_model.encoder.layers.0.self_attn.q_proj.weight torch.Size([1024, 1024])
7 transformer_model.encoder.layers.0.self_attn.q_proj.bias torch.Size([1024])
8 transformer_model.encoder.layers.0.self_attn.out_proj.weight torch.Size([1024, 1024])
9 transformer_model.encoder.layers.0.self_attn.out_proj.bias torch.Size([1024])
10 transformer_model.encoder.layers.0.self_attn_layer_norm.weight torch.Size([1024])
11 transformer_model.encoder.layers.0.self_attn_layer_norm.bias torch.Size([1024])
12 transformer_model.encoder.layers.

In [23]:
sample_input_ids = torch.randint(0, 1000, [8, 248])
sample_attention_mask = torch.randint(0, 1000, [8, 248])

In [24]:
sample_model(sample_input_ids, sample_attention_mask)[1].shape

torch.Size([8, 1024])

In [25]:
torch.sum(torch.randn([8, 496, 768]), axis=1)

tensor([[-21.0677, -42.9958,  14.3378,  ...,  -4.5223,   0.5046,  37.1187],
        [  6.4391,   0.1978,   8.0326,  ..., -29.4494,   5.1051,   7.0165],
        [ 40.7368, -11.5052,  24.5615,  ...,  15.3304, -21.9535, -11.8778],
        ...,
        [ 19.7794,   3.5911, -28.0920,  ..., -44.1329,  44.9046, -11.6836],
        [ 46.3422,  -7.2867, -12.5700,  ..., -15.5036, -19.0379,  16.4539],
        [ 15.3006, -20.9010, -11.6092,  ..., -15.8121,  52.2668, -24.7503]])

### Evaluation and Prediction

In [26]:
def eval_mse(model, data_loader):
    model.eval()
    mse_sum = 0
    mse_loss = nn.MSELoss(reduction='sum')
    
    with torch.no_grad():
        for batch_num, record in enumerate(data_loader):
            input_ids, attention_mask, target = record['input_ids'].to(cfg.DEVICE), record['attention_mask'].to(cfg.DEVICE), record['target'].to(cfg.DEVICE)
            pred, _ = model(input_ids, attention_mask)
            mse_sum += mse_loss(pred.flatten().cpu(), target.cpu())
            
    return mse_sum / len(data_loader.dataset)

In [27]:
def predict(model, data_loader):
    model.eval()
    result = []
    
    with torch.no_grad():
        for batch_num, record in tqdm(enumerate(data_loader), total=len(data_loader)):
            input_ids, attention_mask = record['input_ids'].to(cfg.DEVICE), record['attention_mask'].to(cfg.DEVICE)
            pred, _ = model(input_ids, attention_mask)
            result.extend(pred.flatten().to("cpu").tolist())
            
    return np.array(result)

In [28]:
sample_dl = DataLoader(sample_ds, shuffle=False, batch_size=16, num_workers=1)

### Optimizer and Sampler

In [29]:
def create_optimizer(model, base_lr=5e-5):
    named_parameters = list(model.named_parameters())
    
    regressor_param_start = 515
    attention_start = 511
    roberta_parameters = named_parameters[:attention_start]
    attention_parameters = named_parameters[attention_start:regressor_param_start]
    regressor_parameters = named_parameters[regressor_param_start:]
    
    attention_group = [params for (name, params) in attention_parameters]
    regressor_group = [params for (name, params) in regressor_parameters]
    
    parameters = []
    parameters.append({"params": attention_group, "lr": base_lr * 20})
    parameters.append({"params": regressor_group, "lr": base_lr * 20})
    
    for layer_num, (name, params) in enumerate(roberta_parameters):
        weight_decay = 0.0 if 'bias' in name else 0.01
        
        lr = find_lr_for_layer(layer_num, layer_lrs)
            
        parameters.append({"params": params,
                           "weight_decay": weight_decay,
                           "lr": lr})
        
    return AdamW(parameters)

In [30]:
sample_optimizer = create_optimizer(sample_model)

In [31]:
from torch.utils.data import Sampler,SequentialSampler,RandomSampler,SubsetRandomSampler
from collections import Counter

class WeightedSampler(Sampler):
    
    def __init__(self, dataset):
        
        self.indices = list(range(len(dataset)))
        self.num_samples = len(dataset)
        self.label_to_count = dict(Counter(dataset.bins))
        weights = [1/self.label_to_count[i] for i in dataset.bins]
        
        self.weights = torch.tensor(weights,dtype=torch.double)
        
    def __iter__(self):
        count = 0
        index = [self.indices[i] for i in torch.multinomial(self.weights, self.num_samples, replacement=True)]
        while count < self.num_samples:
            yield index[count]
            count += 1
    
    def __len__(self):
        return self.num_samples

### Training

In [32]:
def choose_eval_period(val_rmse):
    for rmse, period in cfg.EVAL_SCHEDULE:
        if val_rmse >= rmse:
            return period

In [33]:
def serialize_best(best_val_rmse, best_epoch, val_rmse, epoch, model, model_path):
    if not best_val_rmse or val_rmse < best_val_rmse:
        best_val_rmse = val_rmse
        best_epoch = epoch
        if not model_path.parent.exists():
            os.makedirs(model_path.parent)
        
        torch.save(model.state_dict(), model_path)
        print(f"New best_val_rmse: {best_val_rmse:0.4}")
    else:       
        print(f"Still best_val_rmse: {best_val_rmse:0.4}",
              f"(from epoch {best_epoch})")
    return best_epoch, best_val_rmse

In [34]:
class Trainer():
    def __init__(self, model, model_path, train_loader, val_loader, optimizer, scheduler=None, num_epochs=cfg.NUM_EPOCHS):
        self.model, self.model_path, self.train_loader, self.val_loader, self.optimizer, self.scheduler, self.num_epochs = (
            model, model_path, train_loader, val_loader, optimizer, scheduler, num_epochs
        )
            
    def train(self):
        self.model.train()
        mse_loss = nn.MSELoss(reduction='mean')
        
        best_val_rmse = None
        best_epoch = 0
        step = 0
        last_eval_step = 0
        eval_period = cfg.EVAL_SCHEDULE[0][1]    

        start = time.time()
        
        tbar = tqdm(range(self.num_epochs), total=self.num_epochs)
        for epoch in tbar:
            tbar.set_description(f'Epoch: {epoch}')
            val_rmse = None
            for batch_num, record in enumerate(self.train_loader):
                input_ids, attention_mask, target = record['input_ids'].to(cfg.DEVICE), record['attention_mask'].to(cfg.DEVICE), record['target'].to(cfg.DEVICE)
                
                self.optimizer.zero_grad()
                
                pred, _ = self.model(input_ids, attention_mask)
                
                mse = mse_loss(pred.flatten(), target)
                
                mse.backward()
                
                self.optimizer.step()
                if self.scheduler:
                    self.scheduler.step()
                    
                if step >= last_eval_step + eval_period:
                    elapsed_seconds = time.time() - start
                    num_steps = step - last_eval_step
                    print(f"\n{num_steps} steps took {elapsed_seconds:0.3} seconds")
                    last_eval_step = step
                    
                    val_rmse = np.sqrt(eval_mse(self.model, self.val_loader))
                    print(f"Epoch: {epoch} batch_num: {batch_num}", f"val_rmse: {val_rmse:0.4} ", end='')
                    
                    eval_period = choose_eval_period(val_rmse)
                    best_epoch, best_val_rmse = serialize_best(best_val_rmse, best_epoch, val_rmse, epoch, model, self.model_path)
                    start = time.time()
                
                step += 1
        return best_val_rmse

In [35]:
kfold = KFold(n_splits=cfg.NUM_FOLDS, random_state=cfg.SEED, shuffle=True)
splits = kfold.split(train_df)

In [36]:
%%time

list_val_rmse = []

pbar = tqdm(enumerate(splits), total=cfg.NUM_FOLDS, position=0, leave=True)
for fold, (train_indices, val_indices) in pbar:
    pbar.set_description(f'Fold {fold}')
    model_path = cfg.MODEL_FOLDER/f"{cfg.model_name.replace('/', '_')}_{fold + 1}/model_{fold + 1}.pth"
    
    set_random_seed(cfg.SEED + fold)
    
    tokenizer = AutoTokenizer.from_pretrained(cfg.TOKENIZER_PATH)
    
    train_dataset = CommonLitDataset(train_df.loc[train_indices], tokenizer)    
    val_dataset = CommonLitDataset(train_df.loc[val_indices], tokenizer)
    
    train_loader = DataLoader(train_dataset, batch_size=cfg.BATCH_SIZE,
                              drop_last=False, shuffle=True, num_workers=cfg.NUM_WORKERS)    
    val_loader = DataLoader(val_dataset, batch_size=cfg.BATCH_SIZE,
                            drop_last=False, shuffle=False, num_workers=cfg.NUM_WORKERS)
    
    set_random_seed(cfg.SEED + fold)
    
    model = CommonLitModel().to(cfg.DEVICE)
        
    optimizer = create_optimizer(model)
    
    scheduler = get_cosine_schedule_with_warmup(optimizer,
                                                num_training_steps=cfg.NUM_EPOCHS * len(train_loader), 
                                                num_warmup_steps=50)
    
    trainer = Trainer(model, model_path, train_loader, val_loader, optimizer, scheduler = scheduler)
    list_val_rmse.append(trainer.train())
    
    tokenizer.save_pretrained(str(model_path.parent))
    del model
    gc.collect()
    
    if cfg.DEVICE == 'cuda':
        torch.cuda.empty_cache()
        
print("\nPerformance estimates:")
print(list_val_rmse)
print("Mean:", np.array(list_val_rmse).mean())


16 steps took 16.6 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.8931 New best_val_rmse: 0.8931

16 steps took 14.3 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.6515 New best_val_rmse: 0.6515

16 steps took 14.4 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.7985 Still best_val_rmse: 0.6515 (from epoch 0)

16 steps took 14.4 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.7719 Still best_val_rmse: 0.6515 (from epoch 0)

16 steps took 14.4 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.7072 Still best_val_rmse: 0.6515 (from epoch 0)

16 steps took 14.4 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.678 Still best_val_rmse: 0.6515 (from epoch 0)

16 steps took 14.4 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.5962 New best_val_rmse: 0.5962

16 steps took 14.4 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.5374 New best_val_rmse: 0.5374

16 steps took 14.4 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.5383 Still best_val_rmse: 0.5374 (from epoch 0)

16 steps took 14.7 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.5117


16 steps took 16.3 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.7084 New best_val_rmse: 0.7084

16 steps took 14.3 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.6344 New best_val_rmse: 0.6344

16 steps took 14.4 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.7007 Still best_val_rmse: 0.6344 (from epoch 0)

16 steps took 14.4 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.5596 New best_val_rmse: 0.5596

16 steps took 14.4 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.5558 New best_val_rmse: 0.5558

16 steps took 14.4 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.6971 Still best_val_rmse: 0.5558 (from epoch 0)

16 steps took 14.4 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.5417 New best_val_rmse: 0.5417

16 steps took 14.4 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.7419 Still best_val_rmse: 0.5417 (from epoch 0)

16 steps took 14.4 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.5798 Still best_val_rmse: 0.5417 (from epoch 0)

16 steps took 14.7 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.5198 New best_val_rm


16 steps took 16.3 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.9779 New best_val_rmse: 0.9779

16 steps took 14.3 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.7781 New best_val_rmse: 0.7781

16 steps took 14.4 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.6573 New best_val_rmse: 0.6573

16 steps took 14.4 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.6245 New best_val_rmse: 0.6245

16 steps took 14.4 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.6699 Still best_val_rmse: 0.6245 (from epoch 0)

16 steps took 14.4 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.7512 Still best_val_rmse: 0.6245 (from epoch 0)

16 steps took 14.4 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.6194 New best_val_rmse: 0.6194

16 steps took 14.4 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.7121 Still best_val_rmse: 0.6194 (from epoch 0)

16 steps took 14.4 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.5345 New best_val_rmse: 0.5345

16 steps took 14.7 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.504 New best_val_rmse: 0.504

16 step


16 steps took 16.3 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.7563 New best_val_rmse: 0.7563

16 steps took 14.3 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.6404 New best_val_rmse: 0.6404

16 steps took 14.3 seconds
Epoch: 0 batch_num: 48 val_rmse: 1.004 Still best_val_rmse: 0.6404 (from epoch 0)

16 steps took 14.4 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.7435 Still best_val_rmse: 0.6404 (from epoch 0)

16 steps took 14.4 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.6564 Still best_val_rmse: 0.6404 (from epoch 0)

16 steps took 14.4 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.6377 New best_val_rmse: 0.6377

16 steps took 14.4 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.5479 New best_val_rmse: 0.5479

16 steps took 14.4 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.5184 New best_val_rmse: 0.5184

16 steps took 14.3 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.5743 Still best_val_rmse: 0.5184 (from epoch 0)

16 steps took 14.8 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.6278 Still best_val_r


16 steps took 16.3 seconds
Epoch: 0 batch_num: 16 val_rmse: 1.283 New best_val_rmse: 1.283

16 steps took 14.3 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.7121 New best_val_rmse: 0.7121

16 steps took 14.4 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.6553 New best_val_rmse: 0.6553

16 steps took 14.4 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.6183 New best_val_rmse: 0.6183

16 steps took 14.4 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.623 Still best_val_rmse: 0.6183 (from epoch 0)

16 steps took 14.4 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.5669 New best_val_rmse: 0.5669

16 steps took 14.4 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.6269 Still best_val_rmse: 0.5669 (from epoch 0)

16 steps took 14.4 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.7732 Still best_val_rmse: 0.5669 (from epoch 0)

16 steps took 14.4 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.5977 Still best_val_rmse: 0.5669 (from epoch 0)

16 steps took 14.8 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.5378 New best_val_rmse:


16 steps took 16.3 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.7622 New best_val_rmse: 0.7622

16 steps took 14.4 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.785 Still best_val_rmse: 0.7622 (from epoch 0)

16 steps took 14.4 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.6317 New best_val_rmse: 0.6317

16 steps took 14.4 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.6095 New best_val_rmse: 0.6095

16 steps took 14.4 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.5888 New best_val_rmse: 0.5888

16 steps took 14.4 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.5495 New best_val_rmse: 0.5495

16 steps took 14.4 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.5211 New best_val_rmse: 0.5211

16 steps took 14.4 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.7054 Still best_val_rmse: 0.5211 (from epoch 0)

16 steps took 14.4 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.5248 Still best_val_rmse: 0.5211 (from epoch 0)

16 steps took 14.7 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.5044 New best_val_rmse: 0.5044

16 ste

### Verify the model

In [38]:
from sklearn.svm import SVR
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm

In [39]:
cfg.model_offset = 0
cfg.model_limit = 6
cfg.n_folds = 5
cfg.svm_kernels = ['rbf']
cfg.svm_c = 5

In [40]:
num_bins = int(np.ceil(np.log2(len(train_df))))
train_df['bins'] = pd.cut(train_df['target'], bins=num_bins, labels=False)
bins = train_df['bins'].values

In [41]:
%%time

inference_models = []
for i in range(1, cfg.NUM_FOLDS + 1):
    print(f'Model {i}')
    inference_model = CommonLitModel()
    inference_model = inference_model.cuda()
    inference_model.load_state_dict(torch.load(str(MODELS_PATH/f"{cfg.model_name.replace('/', '_')}_{i}/model_{i}.pth")))
    inference_model.eval();
    inference_models.append(inference_model)

Model 1
Model 2
Model 3
Model 4
Model 5
Model 6
CPU times: user 24.9 s, sys: 6.41 s, total: 31.3 s
Wall time: 37.1 s


In [45]:
from transformers import BartTokenizer

tokenizers = []
for i in range(1, cfg.NUM_FOLDS):
    tokenizer = BartTokenizer.from_pretrained(MODELS_PATH/f"{cfg.model_name.replace('/', '_')}_{i}")
    tokenizers.append(tokenizer)

In [46]:
def get_cls_embeddings(dl, transformer_model):
    cls_embeddings = []
    with torch.no_grad():
        for input_features in tqdm(dl, total=len(dl)):
            output, context_vector = transformer_model(input_features['input_ids'].cuda(), input_features['attention_mask'].cuda())
#             cls_embeddings.extend(output['last_hidden_state'][:,0,:].detach().cpu().numpy())
            embedding_out = context_vector.detach().cpu().numpy()
            cls_embeddings.extend(embedding_out)
    return np.array(cls_embeddings)

In [47]:
def rmse_score(X, y):
    return np.sqrt(mean_squared_error(X, y))

In [48]:
def convert_to_list(t):
    return t.flatten().long()

class CommonLitDataset(nn.Module):
    def __init__(self, text, test_id, tokenizer, max_len=128):
        self.excerpt = text
        self.test_id = test_id
        self.max_len = max_len
        self.tokenizer = tokenizer
    
    def __getitem__(self,idx):
        encode = self.tokenizer(self.excerpt[idx],
                                return_tensors='pt',
                                max_length=self.max_len,
                                padding='max_length',
                                truncation=True)
        return {'input_ids': convert_to_list(encode['input_ids']),
                'attention_mask': convert_to_list(encode['attention_mask']),
                'id': self.test_id[idx]}
    
    def __len__(self):
        return len(self.excerpt)

In [49]:
def create_dl(df, tokenizer):
    text = df['excerpt'].values
    ids = df['id'].values
    ds = CommonLitDataset(text, ids, tokenizer, max_len=cfg.MAX_LEN)
    return DataLoader(ds, 
                      batch_size = cfg.BATCH_SIZE,
                      shuffle=False,
                      num_workers = 1,
                      pin_memory=True,
                      drop_last=False
                     )

In [50]:
train_df = pd.read_csv(DATA_PATH/'train-orig.csv')
test_df = pd.read_csv(DATA_PATH/'test.csv')
remove_unnecessary(train_df)

In [51]:
train_target_mean = train_df['target'].mean()
train_target_std = train_df['target'].std()
train_df['normalized_target'] = (train_df['target'] - train_target_mean) / train_target_std

In [52]:
%%time

train_target = train_df['normalized_target'].values

def calc_mean(scores):
    return np.mean(np.array(scores), axis=0)

final_scores = []
final_rmse = []
kernel_rmse_score_mean = []
final_kernel_predictions_means = []
for j, (inference_model, tokenizer) in enumerate(zip(inference_models, tokenizers)):
    print('Model', j)
    test_dl = create_dl(test_df, tokenizer)
    train_dl = create_dl(train_df, tokenizer)
    transformer_model = inference_model
    transformer_model.cuda()
    X = get_cls_embeddings(train_dl, transformer_model)
    
    y = train_target
    X_test = get_cls_embeddings(test_dl, transformer_model)
    
    kfold = StratifiedKFold(n_splits=cfg.NUM_FOLDS)
    scores = []
    rmse_scores = []
    kernel_predictions_means = []
    for kernel in cfg.svm_kernels:
        print('Kernel', kernel)
        kernel_scores = []
        kernel_rmse_scores = []
        kernel_predictions = []
        for k, (train_idx, valid_idx) in enumerate(kfold.split(X, bins)):

            print('Fold', k, train_idx.shape, valid_idx.shape)
            model = SVR(C=cfg.svm_c, kernel=kernel, gamma='auto')

            X_train, y_train = X[train_idx], y[train_idx]
            X_valid, y_valid = X[valid_idx], y[valid_idx]
            model.fit(X_train, y_train)
            prediction = model.predict(X_valid)
            kernel_predictions.append(prediction)
            kernel_rmse_scores.append(rmse_score(prediction, y_valid))
            print('rmse_score', kernel_rmse_scores[k])
            kernel_scores.append(model.predict(X_test))
        kernel_predictions_means.append(np.array([np.mean(kp) for kp in kernel_predictions]).mean())
        scores.append(calc_mean(kernel_scores))
        kernel_rmse_score = calc_mean(kernel_rmse_scores)
        kernel_rmse_score_mean.append(kernel_rmse_score)
        rmse_scores.append(kernel_rmse_score)
    final_kernel_predictions_means.append(kernel_predictions_means)
    final_scores.append(calc_mean(scores))
    final_rmse.append(calc_mean(rmse_scores))
print('FINAL RMSE score', np.mean(np.array(final_rmse)))

Model 0



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.37325547229341144
Fold 1 (2361,) (472,)
rmse_score 0.3453324134543273
Fold 2 (2361,) (472,)
rmse_score 0.32340817929201776
Fold 3 (2361,) (472,)
rmse_score 0.3204405526229345
Fold 4 (2361,) (472,)
rmse_score 0.3333327884678928
Fold 5 (2361,) (472,)
rmse_score 0.3635039683803656
Model 1



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.29010519810262403
Fold 1 (2361,) (472,)
rmse_score 0.3004871401415004
Fold 2 (2361,) (472,)
rmse_score 0.2570034726737771
Fold 3 (2361,) (472,)
rmse_score 0.28763543759620486
Fold 4 (2361,) (472,)
rmse_score 0.2990042489319806
Fold 5 (2361,) (472,)
rmse_score 0.30301432244041115
Model 2



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.31286316098675426
Fold 1 (2361,) (472,)
rmse_score 0.30026259230902386
Fold 2 (2361,) (472,)
rmse_score 0.2755827957132132
Fold 3 (2361,) (472,)
rmse_score 0.29837352871199657
Fold 4 (2361,) (472,)
rmse_score 0.3043165327157811
Fold 5 (2361,) (472,)
rmse_score 0.30457660838784817
Model 3



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.35345980165197854
Fold 1 (2361,) (472,)
rmse_score 0.34255884712955575
Fold 2 (2361,) (472,)
rmse_score 0.31107739678635
Fold 3 (2361,) (472,)
rmse_score 0.3140189767969981
Fold 4 (2361,) (472,)
rmse_score 0.33835151304788896
Fold 5 (2361,) (472,)
rmse_score 0.35750849503225257
Model 4



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.319893525050458
Fold 1 (2361,) (472,)
rmse_score 0.2997590314969981
Fold 2 (2361,) (472,)
rmse_score 0.26195076450751587
Fold 3 (2361,) (472,)
rmse_score 0.2603595665123617
Fold 4 (2361,) (472,)
rmse_score 0.3039593548003887
Fold 5 (2361,) (472,)
rmse_score 0.30584878947201594
FINAL RMSE score 0.31204148251689423
CPU times: user 7min 52s, sys: 6.32 s, total: 7min 58s
Wall time: 5min 49s


In [53]:
final_kernel_predictions_means

[[0.0004524387870961925],
 [-0.0012236217380374081],
 [-0.0014261383614178824],
 [-0.0022271296833980864],
 [0.0007525520826139716]]

In [54]:
# (train_df['target'] - cfg.train_target_mean) / cfg.train_target_std
final_scores_normalized = np.array(final_scores) * train_target_std + train_target_mean

In [55]:
kernel_rmse_score_mean_array = np.array(kernel_rmse_score_mean)
kernel_rmse_score_mean_sum = np.sum(kernel_rmse_score_mean_array)
prop_losses = kernel_rmse_score_mean_array / kernel_rmse_score_mean_sum
prop_losses_sum = (1 - prop_losses).sum()
weights = (1 - prop_losses) / prop_losses_sum
weights

array([0.19500535, 0.20360527, 0.20203695, 0.19613497, 0.20321746])

In [56]:
def calc_mean(scores, weights=weights):
    return np.average(np.array(scores), weights=weights, axis=0)

In [57]:
target_mean = train_df['target'].mean()
final_scores_flat = calc_mean(final_scores_normalized).flatten()
final_scores_mean = final_scores_flat.mean()
target_mean, np.array(final_scores_normalized).mean()
# (-0.9579984513405823, -0.8029817438292849)

(-0.9596573929279916, -0.8822176723123903)

In [58]:
final_scores_flat

array([-0.29396289, -0.54543004, -0.45002452, -2.10472559, -1.79757158,
       -1.30941135,  0.32487005])

In [59]:
mean_diff = target_mean - final_scores_mean
mean_diff, mean_diff / len(final_scores)

(-0.07733511993582376, -0.015467023987164753)

In [60]:
sample_df['target'] = final_scores_flat + mean_diff
# sample_df['target'] = len(final_scores) / np.sum(1 / np.array(final_scores), axis=0) # harmonic mean
sample_df

,id,target
0,c0f722661,-0.371298
1,f0953f0a5,-0.622765
2,0df072751,-0.527360
3,04caf4e0c,-2.182061
4,0e63f8bea,-1.874907
5,12537fe78,-1.386746
6,965e592c0,0.247535


### Prepare Packaging

In [61]:
cfg.model_name

'bart-large'

In [62]:
BEST_MODEL_FOLDER = MODELS_PATH/cfg.model_name/'best'
!rm -rf {BEST_MODEL_FOLDER}
!mkdir -p {BEST_MODEL_FOLDER}

In [63]:
BEST_MODEL_FOLDER

PosixPath('/home/commonlit/models/bart-large/best')

In [64]:
cfg.NUM_FOLDS

6

In [65]:
bestmodels = [MODELS_PATH/f'{cfg.model_name}_{i + 1}' for i in range(0, cfg.NUM_FOLDS)]

In [66]:
bestmodels

[PosixPath('/home/commonlit/models/bart-large_1'),
 PosixPath('/home/commonlit/models/bart-large_2'),
 PosixPath('/home/commonlit/models/bart-large_3'),
 PosixPath('/home/commonlit/models/bart-large_4'),
 PosixPath('/home/commonlit/models/bart-large_5'),
 PosixPath('/home/commonlit/models/bart-large_6')]

In [67]:
from shutil import copyfile

def normalize_name(path_name):
    return path_name.replace('', '')

for i, best_model in enumerate(bestmodels):
    print(f'Processing {i}th model')
    i = i + 1
    best_model_file = f'{best_model}/model_{i}.pth'
    if Path(best_model_file).exists():
        copyfile(best_model_file, f'{BEST_MODEL_FOLDER}/{i}_pytorch_model.bin')
        tokenizer_path = Path(BEST_MODEL_FOLDER/f'tokenizer-{i}')
        tokenizer_path.mkdir(parents=True, exist_ok=True)
        assert tokenizer_path.exists()

        tokenizer_json = Path(normalize_name(f'{MODELS_PATH/cfg.model_name}_{i}/tokenizer_config.json'))
        assert tokenizer_json.exists(), f'{tokenizer_json} does not exist'
        copyfile(tokenizer_json, tokenizer_path/'tokenizer.json')

        vocab_txt = Path(normalize_name(f'{MODELS_PATH/cfg.model_name}_{i}/vocab.json'))
        assert vocab_txt.exists(), f'{vocab_txt} does not exist'
        copyfile(vocab_txt, tokenizer_path/'vocab.json')

        merges = Path(normalize_name(f'{MODELS_PATH/cfg.model_name}_{i}/merges.txt'))
        assert merges.exists()
        copyfile(merges, tokenizer_path/'merges.txt')
    else:
        print(f'{best_model_file} is missing')

Processing 0th model
Processing 1th model
Processing 2th model
Processing 3th model
Processing 4th model
Processing 5th model


In [68]:
import shutil

shutil.make_archive(MODELS_PATH/cfg.model_name/'best_models', 'zip', BEST_MODEL_FOLDER)

'/home/commonlit/models/bart-large/best_models.zip'

In [69]:
!ls {MODELS_PATH/cfg.model_name}

best  best_models.zip


In [70]:
!mv {MODELS_PATH}/{cfg.model_name}.yaml {MODELS_PATH/cfg.model_name}

mv: cannot stat '/home/commonlit/models/bart-large.yaml': No such file or directory


In [71]:
transformer_model.transformer_model.save_pretrained(save_directory=f'{MODELS_PATH/cfg.model_name}/lm')

In [72]:
!du -h {MODELS_PATH/cfg.model_name}/*

1.3M	/home/commonlit/models/bart-large/best/tokenizer-1
1.3M	/home/commonlit/models/bart-large/best/tokenizer-2
1.3M	/home/commonlit/models/bart-large/best/tokenizer-3
1.3M	/home/commonlit/models/bart-large/best/tokenizer-4
1.3M	/home/commonlit/models/bart-large/best/tokenizer-5
1.3M	/home/commonlit/models/bart-large/best/tokenizer-6
9.2G	/home/commonlit/models/bart-large/best
8.4G	/home/commonlit/models/bart-large/best_models.zip
1.6G	/home/commonlit/models/bart-large/lm


In [73]:
shutil.make_archive(MODELS_PATH/cfg.model_name/'lm', 'zip', f'{MODELS_PATH/cfg.model_name}/lm')

'/home/commonlit/models/bart-large/lm.zip'

In [74]:
!kaggle datasets init -p {MODELS_PATH/cfg.model_name}

Data package template written to: /home/commonlit/models/bart-large/dataset-metadata.json


In [75]:
dataset_json_path = Path(MODELS_PATH/cfg.model_name/'dataset-metadata.json')
assert dataset_json_path.exists()

In [76]:
!cat {str(dataset_json_path)}

{
  "title": "INSERT_TITLE_HERE",
  "id": "gilfernandes/INSERT_SLUG_HERE",
  "licenses": [
    {
      "name": "CC0-1.0"
    }
  ]
}

In [77]:
with open(dataset_json_path, 'r') as f:
    dataset_json = f.read()
    dataset_json = dataset_json.replace('INSERT_TITLE_HERE', f'commonlit-{cfg.model_name}-light').replace('INSERT_SLUG_HERE', f'commonlit-{cfg.model_name}-light')
    print(dataset_json)
with(open(dataset_json_path, 'w')) as f:
    f.write(dataset_json)

{
  "title": "commonlit-bart-large-light",
  "id": "gilfernandes/commonlit-bart-large-light",
  "licenses": [
    {
      "name": "CC0-1.0"
    }
  ]
}


In [78]:
!rm -rf {MODELS_PATH/cfg.model_name}/best
!rm -rf {MODELS_PATH/cfg.model_name}/lm

In [80]:
!kaggle datasets create -p {MODELS_PATH/cfg.model_name}

Starting upload for file best_models.zip
100%|██████████████████████████████████████| 8.31G/8.31G [14:19<00:00, 10.4MB/s]
Upload successful: best_models.zip (8GB)
Starting upload for file lm.zip
100%|██████████████████████████████████████| 1.38G/1.38G [03:05<00:00, 7.99MB/s]
Upload successful: lm.zip (1GB)
400 - Bad Request


In [ ]:
!kaggle datasets version -p {MODELS_PATH/cfg.model_name} -m "Version with merges.txt" -d

In [ ]:
state_dict = torch.load(str(MODELS_PATH/f'distilroberta-0/checkpoint-105/pytorch_model.bin'))

In [ ]:
loaded_model = CommonLitModel()

In [ ]:
loaded_model.load_state_dict(state_dict)